# Build an Agent

:::info Prerequisites

This guide assumes familiarity with the following concepts:

- [Chat Models](/docs/concepts/#chat-models)
- [Tools](/docs/concepts/#tools)
- [Agents](/docs/concepts/#agents)

:::

By themselves, language models can't take actions - they just output text.
A big use case for LangChain is creating **agents**.
Agents are systems that use LLMs as reasoning engines to determine which actions to take and the inputs to pass them.
After executing actions, the results can be fed back into the LLM to determine whether more actions are needed, or whether it is okay to finish.

In this tutorial we will build an agent that can interact with a search engine. You will be able to ask this agent questions, watch it call the search tool, and have conversations with it.

## End-to-end agent

The code snippet below represents a fully functional agent that uses an LLM to decide which tools to use. It is equipped with a generic search tool. It has conversational memory - meaning that it can be used as a multi-turn chatbot.

In the rest of the guide, we will walk through the individual components and what each part does - but if you want to just grab some code and get started, feel free to use this!

In [1]:
# Import relevant functionality
from langchain_anthropic import ChatAnthropic
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

# Create the agent
memory = MemorySaver()
model = ChatAnthropic(model_name="claude-3-sonnet-20240229")
search = TavilySearchResults(max_results=2)
tools = [search]
agent_executor = create_react_agent(model, tools, checkpointer=memory)

# Use the agent
config = {"configurable": {"thread_id": "abc123"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob! and i live in sf")]}, config
):
    print(chunk)
    print("----")

for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather where I live?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content="Hello Bob! Since you didn't ask a specific question, I don't need to use any tools for this interaction. It's nice to meet you. San Francisco is a beautiful city with lots to see and do. What are some of your favorite things about living there?", additional_kwargs={}, response_metadata={'id': 'msg_01DUsxCNBqXmWqehdU2GHGbu', 'model': 'claude-3-sonnet-20240229', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 271, 'output_tokens': 58}}, id='run-6f30ad57-10c4-4879-afde-a4f0c3a61270-0', usage_metadata={'input_tokens': 271, 'output_tokens': 58, 'total_tokens': 329})]}}
----


{'agent': {'messages': [AIMessage(content=[{'text': 'Okay, to get accurate weather information for your location in San Francisco, let me use the search tool:', 'type': 'text'}, {'id': 'toolu_012wbZm5DSo7d1ukQa5axHeW', 'input': {'query': 'san francisco weather'}, 'name': 'tavily_search_results_json', 'type': 'tool_use'}], additional_kwargs={}, response_metadata={'id': 'msg_01WdvAPJm97rizxzYXA87YRd', 'model': 'claude-3-sonnet-20240229', 'stop_reason': 'tool_use', 'stop_sequence': None, 'usage': {'input_tokens': 340, 'output_tokens': 82}}, id='run-b40173bd-c8d8-4176-9624-9880ffbe50db-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'san francisco weather'}, 'id': 'toolu_012wbZm5DSo7d1ukQa5axHeW', 'type': 'tool_call'}], usage_metadata={'input_tokens': 340, 'output_tokens': 82, 'total_tokens': 422})]}}
----


{'tools': {'messages': [ToolMessage(content='[{"url": "https://www.weatherapi.com/", "content": "{\'location\': {\'name\': \'San Francisco\', \'region\': \'California\', \'country\': \'United States of America\', \'lat\': 37.78, \'lon\': -122.42, \'tz_id\': \'America/Los_Angeles\', \'localtime_epoch\': 1726098945, \'localtime\': \'2024-09-11 16:55\'}, \'current\': {\'last_updated_epoch\': 1726098300, \'last_updated\': \'2024-09-11 16:45\', \'temp_c\': 22.8, \'temp_f\': 73.0, \'is_day\': 1, \'condition\': {\'text\': \'Partly cloudy\', \'icon\': \'//cdn.weatherapi.com/weather/64x64/day/116.png\', \'code\': 1003}, \'wind_mph\': 18.6, \'wind_kph\': 29.9, \'wind_degree\': 290, \'wind_dir\': \'WNW\', \'pressure_mb\': 1011.0, \'pressure_in\': 29.86, \'precip_mm\': 0.0, \'precip_in\': 0.0, \'humidity\': 64, \'cloud\': 25, \'feelslike_c\': 24.9, \'feelslike_f\': 76.9, \'windchill_c\': 18.4, \'windchill_f\': 65.1, \'heatindex_c\': 18.4, \'heatindex_f\': 65.1, \'dewpoint_c\': 14.8, \'dewpoint_f\'

{'agent': {'messages': [AIMessage(content='Based on the search results, the current weather in San Francisco is partly cloudy with a temperature of around 73°F (23°C). There are some winds at around 18-29 kph from the west-northwest. The humidity is 64% and visibility is good at 16 km.\n\nThe results provide a detailed snapshot of the current weather conditions in San Francisco specifically for your location. Let me know if you need any other details about the weather where you live!', additional_kwargs={}, response_metadata={'id': 'msg_01JVaP8Sgf3auwHafGM31Kaq', 'model': 'claude-3-sonnet-20240229', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 1028, 'output_tokens': 105}}, id='run-6f83a5a8-6dcf-4d64-8204-d3b4a3ac47e2-0', usage_metadata={'input_tokens': 1028, 'output_tokens': 105, 'total_tokens': 1133})]}}
----


## Setup

### Jupyter Notebook

This guide (and most of the other guides in the documentation) uses [Jupyter notebooks](https://jupyter.org/) and assumes the reader is as well. Jupyter notebooks are perfect interactive environments for learning how to work with LLM systems because oftentimes things can go wrong (unexpected output, API down, etc), and observing these cases is a great way to better understand building with LLMs.

This and other tutorials are perhaps most conveniently run in a Jupyter notebook. See [here](https://jupyter.org/install) for instructions on how to install.

### Installation

To install LangChain run:

In [2]:
%pip install -U langchain-community langgraph langchain-anthropic tavily-python langgraph-checkpoint-sqlite

   ━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.1/2.3 MB 4.6 MB/s eta 0:00:01

   ━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.5/2.3 MB 6.9 MB/s eta 0:00:01

   ━━━━━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.7/2.3 MB 8.0 MB/s eta 0:00:01

   ━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━ 1.0/2.3 MB 5.5 MB/s eta 0:00:01

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━╸━━━━━━━━━━━━ 1.6/2.3 MB 7.2 MB/s eta 0:00:01

...

      Successfully uninstalled langchain-0.3.0.dev1


  Attempting uninstall: langchain-community
    Found existing installation: langchain-community 0.3.0.dev1
    Uninstalling langchain-community-0.3.0.dev1:


      Successfully uninstalled langchain-community-0.3.0.dev1


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-benchmarks 0.0.14 requires langchain<0.3.0,>=0.2.7, but you have langchain 0.3.0.dev2 which is incompatible.
langchain-benchmarks 0.0.14 requires langchain-community<0.3,>=0.2, but you have langchain-community 0.3.0.dev2 which is incompatible.
langchain-benchmarks 0.0.14 requires langchain-openai<0.2.0,>=0.1.14, but you have langchain-openai 0.2.0.dev2 which is incompatible.
langchain-aws 0.1.15 requires langchain-core<0.3,>=0.2.29, but you have langchain-core 0.3.0.dev5 which is incompatible.
langchain-huggingface 0.0.3 requires langchain-core<0.3,>=0.1.52, but you have langchain-core 0.3.0.dev5 which is incompatible.
langchain-chroma 0.1.3 requires langchain-core<0.3,>=0.1.40, but you have langchain-core 0.3.0.dev5 which is incompatible.
langchain-together 0.1.5 requires langchain-core<0.3.0,>=0.2.26, 

Note: you may need to restart the kernel to use updated packages.


For more details, see our [Installation guide](/docs/how_to/installation).

### LangSmith

Many of the applications you build with LangChain will contain multiple steps with multiple invocations of LLM calls.
As these applications get more and more complex, it becomes crucial to be able to inspect what exactly is going on inside your chain or agent.
The best way to do this is with [LangSmith](https://smith.langchain.com).

After you sign up at the link above, make sure to set your environment variables to start logging traces:

```shell
export LANGCHAIN_TRACING_V2="true"
export LANGCHAIN_API_KEY="..."
```

Or, if in a notebook, you can set them with:

```python
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()
```

### Tavily

We will be using [Tavily](/docs/integrations/tools/tavily_search) (a search engine) as a tool.
In order to use it, you will need to get and set an API key:

```bash
export TAVILY_API_KEY="..."
```

Or, if in a notebook, you can set it with:

```python
import getpass
import os

os.environ["TAVILY_API_KEY"] = getpass.getpass()
```

## Define tools

We first need to create the tools we want to use. Our main tool of choice will be [Tavily](/docs/integrations/tools/tavily_search) - a search engine. We have a built-in tool in LangChain to easily use Tavily search engine as tool.


In [3]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in SF")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.78, 'lon': -122.42, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1726098811, 'localtime': '2024-09-11 16:53'}, 'current': {'last_updated_epoch': 1726098300, 'last_updated': '2024-09-11 16:45', 'temp_c': 22.8, 'temp_f': 73.0, 'is_day': 1, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind_mph': 18.6, 'wind_kph': 29.9, 'wind_degree': 290, 'wind_dir': 'WNW', 'pressure_mb': 1011.0, 'pressure_in': 29.86, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 64, 'cloud': 25, 'feelslike_c': 24.9, 'feelslike_f': 76.9, 'windchill_c': 18.4, 'windchill_f': 65.1, 'heatindex_c': 18.4, 'heatindex_f': 65.1, 'dewpoint_c': 14.8, 'dewpoint_f': 58.6, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 5.0, 'gust_mph': 27.3, 'gust_kph': 43.9}}"}, {'url': 'https://www.ventusky.com/san-franci

## Using Language Models

Next, let's learn how to use a language model by to call tools. LangChain supports many different language models that you can use interchangably - select the one you want to use below!

```{=mdx}
import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs openaiParams={`model="gpt-4"`} />
```

In [4]:
# | output: false
# | echo: false

from langchain_anthropic import ChatAnthropic

model = ChatAnthropic(model="claude-3-sonnet-20240229")

You can call the language model by passing in a list of messages. By default, the response is a `content` string.

In [5]:
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="hi!")])
response.content

'Hi there! How can I assist you today?'

We can now see what it is like to enable this model to do tool calling. In order to enable that we use `.bind_tools` to give the language model knowledge of these tools

In [6]:
model_with_tools = model.bind_tools(tools)

We can now call the model. Let's first call it with a normal message, and see how it responds. We can look at both the `content` field as well as the `tool_calls` field.

In [7]:
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: Hello!
ToolCalls: []


Now, let's try calling it with some input that would expect a tool to be called.

In [8]:
response = model_with_tools.invoke([HumanMessage(content="What's the weather in SF?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: [{'text': 'To get current weather information for San Francisco, we can use the tavily_search_results_json tool with an appropriate query:', 'type': 'text'}, {'id': 'toolu_017tgYBkVrtb7Szwfw5NzfRR', 'input': {'query': 'san francisco weather'}, 'name': 'tavily_search_results_json', 'type': 'tool_use'}]
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'san francisco weather'}, 'id': 'toolu_017tgYBkVrtb7Szwfw5NzfRR', 'type': 'tool_call'}]


We can see that there's now no text content, but there is a tool call! It wants us to call the Tavily Search tool.

This isn't calling that tool yet - it's just telling us to. In order to actually call it, we'll want to create our agent.

## Create the agent

Now that we have defined the tools and the LLM, we can create the agent. We will be using [LangGraph](/docs/concepts/#langgraph) to construct the agent. 
Currently we are using a high level interface to construct the agent, but the nice thing about LangGraph is that this high-level interface is backed by a low-level, highly controllable API in case you want to modify the agent logic.


Now, we can initialize the agent with the LLM and the tools.

Note that we are passing in the `model`, not `model_with_tools`. That is because `create_react_agent` will call `.bind_tools` for us under the hood.

In [9]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

## Run the agent

We can now run the agent on a few queries! Note that for now, these are all **stateless** queries (it won't remember previous interactions). Note that the agent will return the **final** state at the end of the interaction (which includes any inputs, we will see later on how to get only the outputs).

First up, let's see how it responds when there's no need to call a tool:

In [10]:
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

response["messages"]

[HumanMessage(content='hi!', additional_kwargs={}, response_metadata={}, id='7678e0ad-b6f7-4756-88b8-f5ab2a1b0f08'),
 AIMessage(content='Hello!', additional_kwargs={}, response_metadata={'id': 'msg_01J7VUEPXtcQGgmb5qHrgtjr', 'model': 'claude-3-sonnet-20240229', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 264, 'output_tokens': 5}}, id='run-fbda63c9-e2a8-46ce-8f0b-b85fbeaafced-0', usage_metadata={'input_tokens': 264, 'output_tokens': 5, 'total_tokens': 269})]

In order to see exactly what is happening under the hood (and to make sure it's not calling a tool) we can take a look at the [LangSmith trace](https://smith.langchain.com/public/28311faa-e135-4d6a-ab6b-caecf6482aaa/r)

Let's now try it out on an example where it should be invoking the tool

In [11]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
)
response["messages"]

[HumanMessage(content='whats the weather in sf?', additional_kwargs={}, response_metadata={}, id='dd268fda-5e57-4bc9-a0a5-63d9e88d7659'),
 AIMessage(content=[{'text': 'To get current weather information for San Francisco, we can use the tavily_search_results_json tool:', 'type': 'text'}, {'id': 'toolu_01FsvX6nE7wAgnuHZoWwEEDC', 'input': {'query': 'san francisco weather'}, 'name': 'tavily_search_results_json', 'type': 'tool_use'}], additional_kwargs={}, response_metadata={'id': 'msg_019wc7ZgXtnqdUGavis6kNdE', 'model': 'claude-3-sonnet-20240229', 'stop_reason': 'tool_use', 'stop_sequence': None, 'usage': {'input_tokens': 269, 'output_tokens': 84}}, id='run-1f61d7a2-6200-4ed3-82d0-90e17e558ca4-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'san francisco weather'}, 'id': 'toolu_01FsvX6nE7wAgnuHZoWwEEDC', 'type': 'tool_call'}], usage_metadata={'input_tokens': 269, 'output_tokens': 84, 'total_tokens': 353}),
 ToolMessage(content='[{"url": "https://www.weatherapi.co

We can check out the [LangSmith trace](https://smith.langchain.com/public/f520839d-cd4d-4495-8764-e32b548e235d/r) to make sure it's calling the search tool effectively.

## Streaming Messages

We've seen how the agent can be called with `.invoke` to get back a final response. If the agent is executing multiple steps, that may take a while. In order to show intermediate progress, we can stream back messages as they occur.

In [12]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content=[{'id': 'toolu_01QGkj39Dk67gbximVZPUbYL', 'input': {'query': 'weather san francisco'}, 'name': 'tavily_search_results_json', 'type': 'tool_use'}], additional_kwargs={}, response_metadata={'id': 'msg_01MbDutXi5oeszaLfdURh5hm', 'model': 'claude-3-sonnet-20240229', 'stop_reason': 'tool_use', 'stop_sequence': None, 'usage': {'input_tokens': 269, 'output_tokens': 60}}, id='run-4e7002fd-8735-41db-b6ed-35c9d3b89ba2-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather san francisco'}, 'id': 'toolu_01QGkj39Dk67gbximVZPUbYL', 'type': 'tool_call'}], usage_metadata={'input_tokens': 269, 'output_tokens': 60, 'total_tokens': 329})]}}
----


{'tools': {'messages': [ToolMessage(content='[{"url": "https://www.weatherapi.com/", "content": "{\'location\': {\'name\': \'San Francisco\', \'region\': \'California\', \'country\': \'United States of America\', \'lat\': 37.78, \'lon\': -122.42, \'tz_id\': \'America/Los_Angeles\', \'localtime_epoch\': 1726098811, \'localtime\': \'2024-09-11 16:53\'}, \'current\': {\'last_updated_epoch\': 1726098300, \'last_updated\': \'2024-09-11 16:45\', \'temp_c\': 22.8, \'temp_f\': 73.0, \'is_day\': 1, \'condition\': {\'text\': \'Partly cloudy\', \'icon\': \'//cdn.weatherapi.com/weather/64x64/day/116.png\', \'code\': 1003}, \'wind_mph\': 18.6, \'wind_kph\': 29.9, \'wind_degree\': 290, \'wind_dir\': \'WNW\', \'pressure_mb\': 1011.0, \'pressure_in\': 29.86, \'precip_mm\': 0.0, \'precip_in\': 0.0, \'humidity\': 64, \'cloud\': 25, \'feelslike_c\': 24.9, \'feelslike_f\': 76.9, \'windchill_c\': 18.4, \'windchill_f\': 65.1, \'heatindex_c\': 18.4, \'heatindex_f\': 65.1, \'dewpoint_c\': 14.8, \'dewpoint_f\'

{'agent': {'messages': [AIMessage(content="The search results provide current weather conditions and forecasts for San Francisco, California. Some key details:\n\n- The current temperature in San Francisco is around 73°F (23°C) with partly cloudy skies.\n\n- Wind speeds are around 18-27 mph (29-44 kph) from the west-northwest. \n\n- Humidity is 64% and visibility is good at 9 miles (16 km).\n\n- Over the next couple weeks, temperatures are expected to stay mild, ranging from highs around 72°F (22°C) to lows in the mid 50sF (12-13°C).\n\n- No rain is in the current forecast.\n\nSo in summary, it's a pleasant partly cloudy day in San Francisco with seasonable temperatures for September and breezy conditions. The weather looks to remain nice over the next couple weeks as well with no rain expected.", additional_kwargs={}, response_metadata={'id': 'msg_01NhPksUogasTj2gjJnDq9fP', 'model': 'claude-3-sonnet-20240229', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens':

## Streaming tokens

In addition to streaming back messages, it is also useful to be streaming back tokens.
We can do this with the `.astream_events` method.

:::{.callout-important}
This `.astream_events` method only works with Python 3.11 or higher.
:::

In [13]:
async for event in agent_executor.astream_events(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}, version="v1"
):
    kind = event["event"]
    if kind == "on_chain_start":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print(
                f"Starting agent: {event['name']} with input: {event['data'].get('input')}"
            )
    elif kind == "on_chain_end":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print()
            print("--")
            print(
                f"Done agent: {event['name']} with output: {event['data'].get('output')['output']}"
            )
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            # Empty content in the context of OpenAI means
            # that the model is asking for a tool to be invoked.
            # So we only print non-empty content
            print(content, end="|")
    elif kind == "on_tool_start":
        print("--")
        print(
            f"Starting tool: {event['name']} with inputs: {event['data'].get('input')}"
        )
    elif kind == "on_tool_end":
        print(f"Done tool: {event['name']}")
        print(f"Tool output was: {event['data'].get('output')}")
        print("--")

[{'text': 'To', 'type': 'text', 'index': 0}]|[{'text': ' get the current weather', 'type': 'text', 'index': 0}]|[{'text': ' in San Francisco,', 'type': 'text', 'index': 0}]|[{'text': ' we', 'type': 'text', 'index': 0}]|[{'text': ' can use the', 'type': 'text', 'index': 0}]|[{'text': ' Tavily search engine', 'type': 'text', 'index': 0}]|[{'text': ':', 'type': 'text', 'index': 0}]|

[{'id': 'toolu_017QTrz3k83pjEtfWSfQjkL4', 'input': {}, 'name': 'tavily_search_results_json', 'type': 'tool_use', 'index': 1}]|[{'partial_json': '', 'type': 'tool_use', 'index': 1}]|

[{'partial_json': '{"query": "s', 'type': 'tool_use', 'index': 1}]|[{'partial_json': 'an fr', 'type': 'tool_use', 'index': 1}]|[{'partial_json': 'anci', 'type': 'tool_use', 'index': 1}]|[{'partial_json': 'sc', 'type': 'tool_use', 'index': 1}]|[{'partial_json': 'o wea', 'type': 'tool_use', 'index': 1}]|[{'partial_json': 'the', 'type': 'tool_use', 'index': 1}]|[{'partial_json': 'r"}', 'type': 'tool_use', 'index': 1}]|--
Starting tool: tavily_search_results_json with inputs: {'query': 'san francisco weather'}


Done tool: tavily_search_results_json
Tool output was: content='[{"url": "https://www.weatherapi.com/", "content": "{\'location\': {\'name\': \'San Francisco\', \'region\': \'California\', \'country\': \'United States of America\', \'lat\': 37.78, \'lon\': -122.42, \'tz_id\': \'America/Los_Angeles\', \'localtime_epoch\': 1726098992, \'localtime\': \'2024-09-11 16:56\'}, \'current\': {\'last_updated_epoch\': 1726098300, \'last_updated\': \'2024-09-11 16:45\', \'temp_c\': 22.8, \'temp_f\': 73.0, \'is_day\': 1, \'condition\': {\'text\': \'Partly cloudy\', \'icon\': \'//cdn.weatherapi.com/weather/64x64/day/116.png\', \'code\': 1003}, \'wind_mph\': 18.6, \'wind_kph\': 29.9, \'wind_degree\': 290, \'wind_dir\': \'WNW\', \'pressure_mb\': 1011.0, \'pressure_in\': 29.86, \'precip_mm\': 0.0, \'precip_in\': 0.0, \'humidity\': 64, \'cloud\': 25, \'feelslike_c\': 24.9, \'feelslike_f\': 76.9, \'windchill_c\': 18.4, \'windchill_f\': 65.1, \'heatindex_c\': 18.4, \'heatindex_f\': 65.1, \'dewpoint_c\': 1

[{'text': '\n\nThe', 'type': 'text', 'index': 0}]|[{'text': ' search', 'type': 'text', 'index': 0}]|[{'text': ' results provide', 'type': 'text', 'index': 0}]|[{'text': ' details on the current', 'type': 'text', 'index': 0}]|[{'text': ' weather conditions in San', 'type': 'text', 'index': 0}]|[{'text': ' Francisco, including temperature', 'type': 'text', 'index': 0}]|[{'text': ', clou', 'type': 'text', 'index': 0}]|[{'text': 'd cover, wind,', 'type': 'text', 'index': 0}]|

...

[{'text': 'NW\n-', 'type': 'text', 'index': 0}]|[{'text': ' Humidity: ', 'type': 'text', 'index': 0}]|[{'text': '64%\n-', 'type': 'text', 'index': 0}]|[{'text': ' Clou', 'type': 'text', 'index': 0}]|

[{'text': 'd Cover', 'type': 'text', 'index': 0}]|[{'text': ': 25%', 'type': 'text', 'index': 0}]|[{'text': '\n\nSo', 'type': 'text', 'index': 0}]|[{'text': ' in summary, it', 'type': 'text', 'index': 0}]|[{'text': "'s", 'type': 'text', 'index': 0}]|[{'text': ' a', 'type': 'text', 'index': 0}]|[{'text': ' warm', 'type': 'text', 'index': 0}]|

[{'text': ' partly', 'type': 'text', 'index': 0}]|[{'text': ' cloudy day in', 'type': 'text', 'index': 0}]|[{'text': ' San Francisco with moderate', 'type': 'text', 'index': 0}]|[{'text': ' winds and humidity levels', 'type': 'text', 'index': 0}]|

[{'text': '. The temperature', 'type': 'text', 'index': 0}]|[{'text': ' is around 73', 'type': 'text', 'index': 0}]|[{'text': '°F/', 'type': 'text', 'index': 0}]|[{'text': '23°C as', 'type': 'text', 'index': 0}]|[{'text': ' of late', 'type': 'text', 'index': 0}]|[{'text': ' afternoon', 'type': 'text', 'index': 0}]|[{'text': '. Let me know', 'type': 'text', 'index': 0}]|[{'text': ' if you need any', 'type': 'text', 'index': 0}]|

[{'text': ' other', 'type': 'text', 'index': 0}]|[{'text': ' details on', 'type': 'text', 'index': 0}]|[{'text': ' the SF', 'type': 'text', 'index': 0}]|[{'text': ' weather!', 'type': 'text', 'index': 0}]|

## Adding in memory

As mentioned earlier, this agent is stateless. This means it does not remember previous interactions. To give it memory we need to pass in a checkpointer. When passing in a checkpointer, we also have to pass in a `thread_id` when invoking the agent (so it knows which thread/conversation to resume from).

In [14]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [15]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

In [16]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob!")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content="Hello Bob! It's nice to meet you. I don't actually need to use any tools to respond to a simple greeting. How are you doing today?", additional_kwargs={}, response_metadata={'id': 'msg_011osVvD1gEohDvVLUGajUyB', 'model': 'claude-3-sonnet-20240229', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 266, 'output_tokens': 35}}, id='run-9802dd41-050c-47e5-b624-c098b4b53c58-0', usage_metadata={'input_tokens': 266, 'output_tokens': 35, 'total_tokens': 301})]}}
----


In [17]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='You said your name is Bob in your first message when you said "hi im bob!".', additional_kwargs={}, response_metadata={'id': 'msg_01L1AhTC4RkjevWQ2ktPjKkp', 'model': 'claude-3-sonnet-20240229', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 309, 'output_tokens': 22}}, id='run-f407156d-184c-499c-8207-bbc1786dc878-0', usage_metadata={'input_tokens': 309, 'output_tokens': 22, 'total_tokens': 331})]}}
----


Example [LangSmith trace](https://smith.langchain.com/public/fa73960b-0f7d-4910-b73d-757a12f33b2b/r)

If I want to start a new conversation, all I have to do is change the `thread_id` used

In [18]:
config = {"configurable": {"thread_id": "xyz123"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content="Unfortunately, I don't have enough information to know your name. As an AI assistant without personal knowledge about you, I don't have a way to look up or determine your name unless you provide it to me.", additional_kwargs={}, response_metadata={'id': 'msg_01Pgype414fuF8cHzBu5mCsd', 'model': 'claude-3-sonnet-20240229', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 267, 'output_tokens': 46}}, id='run-bbade9ec-ca90-4f1f-bb58-c760e5c0940c-0', usage_metadata={'input_tokens': 267, 'output_tokens': 46, 'total_tokens': 313})]}}
----


## Conclusion

That's a wrap! In this quick start we covered how to create a simple agent. 
We've then shown how to stream back a response - not only the intermediate steps, but also tokens!
We've also added in memory so you can have a conversation with them.
Agents are a complex topic, and there's lot to learn! 

For more information on Agents, please check out the [LangGraph](/docs/concepts/#langgraph) documentation. This has it's own set of concepts, tutorials, and how-to guides.